In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import Toast

In [2]:
# load dataset and bring it in format for toast
network = RoadNetwork()
network.load("../osm_data/porto")
df = pd.read_csv("../datasets/trajectories/Porto/road_segment_map_final.csv", sep=";", usecols=["id", "cpath"])

In [3]:
print(network.gdf_nodes.shape)
#dG = nx.DiGraph(network.G)
dG = nx.line_graph(network.G, create_using=nx.DiGraph)
index = np.arange(2, len(list(dG.nodes))+2) # index shifted, since 0 and 1 are reserved for mask pad tokens
nx.set_node_attributes(dG, dict(zip(list(dG.nodes), index)), name="index")


(5358, 6)


In [51]:
nodes = list(dG.nodes(data=True))

for n, d in nodes:
    print(network.gdf_edges.loc[n, :]["highway"])
    break


motorway_link


In [4]:
torch.cuda.is_available()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
toast = Toast(dG, network.gdf_edges)
toast.train(num_walks=20)

100%|██████████| 11331/11331 [02:19<00:00, 81.46it/s]
/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/toast.py:54: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/torch/csrc/utils/tensor_new.cpp:210.)
  batch_types = torch.Tensor(batch_types).to(self.device)


Epoch: 0, iter 5000, total loss: 0.8914021849632263, context loss: 0.8914021849632263, type loss: 0.0
Epoch: 0, iter 10000, total loss: 0.8610518574714661, context loss: 0.8610518574714661, type loss: 0.0
Epoch: 0, iter 15000, total loss: 0.8426587581634521, context loss: 0.8426587581634521, type loss: 0.0
Epoch: 0, iter 20000, total loss: 0.8284405469894409, context loss: 0.8284405469894409, type loss: 0.0
Epoch: 0, iter 25000, total loss: 0.8080103397369385, context loss: 0.8080103397369385, type loss: 0.0
Epoch: 0, iter 30000, total loss: 0.9491406083106995, context loss: 0.9491406083106995, type loss: 0.0
Epoch: 0, iter 35000, total loss: 0.8777050971984863, context loss: 0.8777050971984863, type loss: 0.0
Epoch: 0, iter 40000, total loss: 1.0736680030822754, context loss: 0.9811930656433105, type loss: 0.09247489273548126
Epoch: 0, iter 45000, total loss: 0.7762971520423889, context loss: 0.7762971520423889, type loss: 0.0
Epoch: 0, iter 50000, total loss: 0.808716356754303, conte

  1%|▏         | 109/11331 [00:01<02:21, 79.21it/s]

KeyboardInterrupt: 